# 0. Imports

In [1]:
import random

import numpy as np
import pandas as pd

from extrucal.extrusion import throughput_cal

from sklearn.utils import shuffle

random.seed(0)

# 1. Dataset Generation using `extrucal`

## 1.1 Preparation of Lists for Arguments

In [2]:
extruder_size = [size for size in range(20, 251, 10)]
extruder_size_df = pd.DataFrame({"extruder_size": extruder_size})
len(extruder_size)

24

In [3]:
metering_depth_percent = [
    np.round(percent, 2) for percent in np.arange(0.02, 0.1, 0.01)
]
metering_depth_percent_df = pd.DataFrame(
    {"metering_depth_percent": metering_depth_percent}
)
len(metering_depth_percent)

8

In [4]:
polymer_density = [density for density in range(800, 1500, 100)]
polymer_density_df = pd.DataFrame({"polymer_density": polymer_density})
len(polymer_density)

7

In [5]:
screw_pitch_percent = [np.round(pitch, 2) for pitch in np.arange(0.6, 2, 0.2)]
screw_pitch_percent_df = pd.DataFrame({"screw_pitch_percent": screw_pitch_percent})
len(screw_pitch_percent)

7

In [6]:
flight_width_percent = [np.round(percent, 2) for percent in np.arange(0.06, 0.2, 0.02)]
flight_width_percent_df = pd.DataFrame({"flight_width_percent": flight_width_percent})
len(flight_width_percent)

8

In [7]:
number_flight = [1, 2]
number_flight_df = pd.DataFrame({"number_flight": number_flight})
len(number_flight)

2

In [8]:
rpm = [r for r in range(1, 101, 10)]
rpm_df = pd.DataFrame({"rpm": rpm})
len(rpm)

10

## 1.2. Preparation of Dataframe

In [9]:
df = (
    extruder_size_df.join(metering_depth_percent_df, how="cross")
    .join(polymer_density_df, how="cross")
    .join(screw_pitch_percent_df, how="cross")
    .join(flight_width_percent_df, how="cross")
    .join(number_flight_df, how="cross")
    .join(rpm_df, how="cross")
)

In [10]:
len(df)

1505280

In [11]:
df["metering_depth"] = df["extruder_size"] * df["metering_depth_percent"]
df["screw_pitch"] = df["extruder_size"] * df["screw_pitch_percent"]
df["flight_width"] = df["extruder_size"] * df["flight_width_percent"]

In [12]:
new_col_order = [
    "extruder_size",
    "metering_depth",
    "polymer_density",
    "rpm",
    "screw_pitch",
    "flight_width",
    "number_flight",
]

df = df[new_col_order]
df.head()

,extruder_size,metering_depth,polymer_density,rpm,screw_pitch,flight_width,number_flight
0,20,0.4,800,1,12.0,1.2,1
1,20,0.4,800,11,12.0,1.2,1
2,20,0.4,800,21,12.0,1.2,1
3,20,0.4,800,31,12.0,1.2,1
4,20,0.4,800,41,12.0,1.2,1


In [13]:
%%time

df.loc[:, "throughput"] = df.apply(
    lambda row: throughput_cal(
        row["extruder_size"],
        row["metering_depth"],
        row["polymer_density"],
        row["rpm"],
        row["screw_pitch"],
        row["flight_width"],
        int(row["number_flight"]),
    )*random.uniform(0.95, 1.05),  # +/- 5% randomness was added to the calculated throughput
    axis=1,
)

CPU times: total: 8min 21s
Wall time: 8min 21s


In [14]:
df

,extruder_size,metering_depth,polymer_density,rpm,screw_pitch,flight_width,number_flight,throughput
0,20,0.4,800,1,12.0,1.2,1,0.006318
1,20,0.4,800,11,12.0,1.2,1,0.068922
2,20,0.4,800,21,12.0,1.2,1,0.127251
3,20,0.4,800,31,12.0,1.2,1,0.184786
4,20,0.4,800,41,12.0,1.2,1,0.250714
...,...,...,...,...,...,...,...,...
1505275,250,22.5,1400,51,450.0,50.0,2,8322.002616
1505276,250,22.5,1400,61,450.0,50.0,2,10012.362935
1505277,250,22.5,1400,71,450.0,50.0,2,11483.210639
1505278,250,22.5,1400,81,450.0,50.0,2,13878.255253


In [15]:
df = shuffle(df)

In [16]:
df

,extruder_size,metering_depth,polymer_density,rpm,screw_pitch,flight_width,number_flight,throughput
727884,130,7.8,1300,41,234.0,13.0,1,795.947880
1325210,230,6.9,800,1,184.0,32.2,2,12.391977
1383377,240,4.8,1100,71,192.0,14.4,2,1289.777079
950997,170,5.1,1000,71,102.0,27.2,2,232.970733
1486160,250,17.5,1100,1,450.0,35.0,1,126.518677
...,...,...,...,...,...,...,...,...
395230,80,3.2,1000,1,144.0,6.4,2,2.272184
692055,130,2.6,900,51,234.0,13.0,2,217.158551
1393001,240,7.2,1200,11,384.0,24.0,1,582.806868
1190605,200,18.0,1400,51,120.0,20.0,1,2133.713332


## 1.3. Save as CSV file

In [17]:
df.to_csv("../data/dataset_generated_by_extrucal_from_1_rpm.csv", index=False)